In [2]:
!pip install pandas


  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -------------- ------------------------- 4.2/11.5 MB 21.9 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 33.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd

# Change this path to match your folder
path = r"C:\3CSE80surya\NM project\cv-corpus-21.0-delta-2025-03-14\en"
tsv_path = path + r"\validated.tsv"

df = pd.read_csv(tsv_path, sep='\t')
print(df.head())

                                           client_id  \
0  116398939d6be70fc5fb532924a130c0adf286ac283499...   
1  24a4da2e8f053a45a0715849c222a40a4b0da9872efb2e...   
2  30849595699bc853c3810a78448acede46888b4e2d0809...   
3  42d53f34c1bc50f7a7c4ed1765a8d1ffeaf5cd441513cc...   
4  436b9e1f9da710d74eb01209f8f269bee70e93cadf2053...   

                           path  \
0  common_voice_en_41923025.mp3   
1  common_voice_en_42356358.mp3   
2  common_voice_en_42165090.mp3   
3  common_voice_en_41921729.mp3   
4  common_voice_en_42528393.mp3   

                                         sentence_id  \
0  f5a2a431746c5229ab696ba0e1a518fe7b26e208ff3b84...   
1  f6f009587d8812c147af1cc05079e1fcd8120c8a98cdf8...   
2  f69afa5e77812e8be0085c874d2a9767323c78ffb43ba6...   
3  f5739acbefdbd3aac990792966fac4d40dcb39eb8dfa21...   
4  f7d35c60d76f025c45a9495757d1ee0e2b7c206317a288...   

                                            sentence sentence_domain  \
0  He was born at Wichenford, in Worcesters

In [5]:
print(df.columns)

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment'],
      dtype='object')


In [6]:
# Use the correct column names based on your dataset
df = df[['path', 'sentence', 'accents', 'age', 'gender']]
df = df.dropna(subset=['sentence'])  # Drop rows with missing transcripts

# Save the cleaned metadata
df.to_csv(r"C:\3CSE80surya\NM project\processed_metadata.csv", index=False)


In [7]:
import os
import librosa
import soundfile as sf

src_dir = os.path.join(path, "clips")
dst_dir = r"C:\3CSE80surya\NM project\processed_wav"
os.makedirs(dst_dir, exist_ok=True)

for filename in df['path']:
    mp3_path = os.path.join(src_dir, filename)
    wav_path = os.path.join(dst_dir, filename.replace(".mp3", ".wav"))

    try:
        y, sr = librosa.load(mp3_path, sr=16000)  # Resample to 16kHz
        sf.write(wav_path, y, 16000)
    except Exception as e:
        print(f"Failed to process {filename}: {e}")


In [8]:
df['wav_path'] = df['path'].apply(lambda x: os.path.join(dst_dir, x.replace(".mp3", ".wav")))
df.to_csv(r"C:\3CSE80surya\NM project\processed_metadata_with_paths.csv", index=False)


In [10]:
# Use the correct file path for processed metadata or augmented metadata (if you already have augmented data)
metadata_path = r"C:\3CSE80surya\NM project\processed_metadata.csv"  # Or use augmented if that's your case
df = pd.read_csv(metadata_path)

# Proceed with the rest of the pipeline...


In [17]:
import os
import librosa
import soundfile as sf
import numpy as np

# Directory paths
src_dir = r"C:\3CSE80surya\NM project\processed_wav"  # Your original .wav files
dst_dir = r"C:\3CSE80surya\NM project\augmented_wav"  # Output folder for augmented files
os.makedirs(dst_dir, exist_ok=True)

# Define noise function (add random noise)
def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

# Loop through all .wav files and apply augmentations
for filename in os.listdir(src_dir):
    if filename.endswith(".wav"):
        wav_path = os.path.join(src_dir, filename)
        
        # Load the audio file
        y, sr = librosa.load(wav_path, sr=16000)  # Ensure it's 16kHz
        
        # Apply noise augmentation (50% chance)
        if np.random.rand() > 0.5:
            y = add_noise(y)

        # Apply pitch shift augmentation (50% chance)
        if np.random.rand() > 0.5:
            y = librosa.effects.pitch_shift(y, sr=sr, n_steps=2)  # Pitch shift with correct arguments

        # Save the augmented file
        augmented_path = os.path.join(dst_dir, filename)
        sf.write(augmented_path, y, sr)

print("Audio augmentation completed!")


Audio augmentation completed!


In [22]:
import pandas as pd

# Load the metadata
metadata_df = pd.read_csv(r"C:\3CSE80surya\NM project\processed_metadata.csv")

# Check the columns
print(metadata_df.columns)


Index(['path', 'sentence', 'accents', 'age', 'gender'], dtype='object')


In [23]:
import pandas as pd

# Load the metadata
metadata_df = pd.read_csv(r"C:\3CSE80surya\NM project\processed_metadata.csv")

# Since the 'path' column already contains the paths, you can directly save it as is
processed_metadata_with_paths_path = r"C:\3CSE80surya\NM project\processed_metadata_with_paths.csv"
metadata_df.to_csv(processed_metadata_with_paths_path, index=False)

print(f"Processed metadata with paths saved to {processed_metadata_with_paths_path}")


Processed metadata with paths saved to C:\3CSE80surya\NM project\processed_metadata_with_paths.csv


In [24]:
import os
import pandas as pd

# Paths
augmented_wav_dir = r"C:\3CSE80surya\NM project\augmented_wav"  # Directory where augmented .wav files are stored
processed_paths_metadata_path = r"C:\3CSE80surya\NM project\processed_metadata_with_paths.csv"  # Metadata file with paths

# Load the processed metadata with paths
processed_paths_df = pd.read_csv(processed_paths_metadata_path)

# List of augmented files in the augmented_wav directory
augmented_files = os.listdir(augmented_wav_dir)

# Prepare a list to store augmented metadata
augmented_metadata = []

for filename in augmented_files:
    # Extract sentence_id from filename (assuming filename corresponds to sentence_id)
    sentence_id = filename.split('.')[0]  # Adjust if needed
    
    # Find the corresponding row from original metadata (based on sentence_id)
    row = processed_paths_df[processed_paths_df['path'].str.contains(sentence_id)].iloc[0]
    
    # Add new row for augmented metadata
    augmented_metadata.append({
        'path': os.path.join(augmented_wav_dir, filename),
        'sentence_id': sentence_id,
        'sentence': row['sentence'],
        'age': row['age'],
        'gender': row['gender'],
        'accent': row['accents']
    })

# Create DataFrame from augmented metadata
augmented_metadata_df = pd.DataFrame(augmented_metadata)

# Save augmented metadata to CSV
augmented_metadata_df.to_csv(r"C:\3CSE80surya\NM project\augmented_metadata.csv", index=False)

print("Augmented metadata saved to augmented_metadata.csv!")


Augmented metadata saved to augmented_metadata.csv!


In [25]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

# Paths
augmented_wav_dir = r"C:\3CSE80surya\NM project\augmented_wav"  # Directory where augmented .wav files are stored
processed_metadata_path = r"C:\3CSE80surya\NM project\augmented_metadata.csv"  # Augmented metadata with paths

# Load augmented metadata
augmented_metadata_df = pd.read_csv(processed_metadata_path)

# Prepare a list to store extracted features
extracted_features = []

# Function to extract MFCC features from audio
def extract_mfcc(file_path):
    # Load audio file with librosa
    y, sr = librosa.load(file_path, sr=16000)  # Resample to 16kHz if needed
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    # Return the mean of MFCCs across time frames as a feature vector
    return np.mean(mfcc.T, axis=0)

# Extract features from both augmented and original files
for index, row in tqdm(augmented_metadata_df.iterrows(), total=augmented_metadata_df.shape[0]):
    file_path = row['path']
    
    try:
        # Extract MFCC features for each file
        mfcc_features = extract_mfcc(file_path)
        
        # Store the features along with metadata
        extracted_features.append({
            'path': file_path,
            'sentence': row['sentence'],
            'age': row['age'],
            'gender': row['gender'],
            'accent': row['accent'],
            'mfcc': mfcc_features
        })
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Convert features to a DataFrame
features_df = pd.DataFrame(extracted_features)

# Save the features to a CSV file for later use
features_df.to_csv(r"C:\3CSE80surya\NM project\extracted_features.csv", index=False)

print("Feature extraction completed and saved to extracted_features.csv!")


100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [00:04<00:00, 57.17it/s]

Feature extraction completed and saved to extracted_features.csv!


In [3]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Alternative model: Trained on English with fewer loading issues
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")


In [6]:
import os

# Path to check
file_path = r'C:\3CSE80surya\NM project\processed_metadata_with_paths.csv'

if os.path.exists(file_path):
    print(f"{file_path} exists!")
else:
    print(f"{file_path} not found.")



C:\3CSE80surya\NM project\processed_metadata_with_paths.csv exists!


In [7]:
import pandas as pd

# Load the processed metadata with paths
file_path = r'C:\3CSE80surya\NM project\processed_metadata_with_paths.csv'
df = pd.read_csv(file_path)

# Check the first few rows to ensure it has the correct structure
print(df.head())


                           path  \
0  common_voice_en_41923025.mp3   
1  common_voice_en_42356358.mp3   
2  common_voice_en_42165090.mp3   
3  common_voice_en_41921729.mp3   
4  common_voice_en_42528393.mp3   

                                            sentence                accents  \
0  He was born at Wichenford, in Worcestershire, ...  United States English   
1  The Portuguese division was overrun and withdr...  United States English   
2            Her health by this stage was also poor.                    NaN   
3  His sporting interests outside of cricket incl...        England English   
4  The following year he was elected to be part o...  United States English   

        age gender  
0  thirties    NaN  
1     teens    NaN  
2       NaN    NaN  
3  nineties    NaN  
4     teens    NaN  


In [9]:
# Update CSV with correct paths
df['path'] = df['path'].apply(lambda x: r'C:\3CSE80surya\NM project\processed_wav\\' + os.path.basename(x))

# Save the updated CSV with correct paths
df.to_csv(r'C:\3CSE80surya\NM project\processed_metadata_with_paths.csv', index=False)


In [11]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="librosa")


In [13]:
import os

# Make sure to handle paths correctly
audio_path = os.path.join('C:\\3CSE80surya\\NM project\\processed_wav', 'common_voice_en_42221077.mp3')


In [14]:
if os.path.exists(audio_path):
    # Process the file
    speech, _ = librosa.load(audio_path, sr=16000)
else:
    print(f"File not found: {audio_path}")


File not found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221077.mp3


In [15]:
print(f"Checking for file: {audio_path}")


Checking for file: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221077.mp3


In [16]:
print("File path:", audio_path)  # Debugging step to see full path.


File path: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221077.mp3


In [18]:
import os

processed_wav_path = 'C:\\3CSE80surya\\NM project\\processed_wav'

# Example files list
files_to_process = ['common_voice_en_42221077.mp3', 'common_voice_en_42221538.mp3']

for file_name in files_to_process:
    audio_path = os.path.join(processed_wav_path, file_name)
    
    print(f"Checking for file: {audio_path}")  # Debugging step
    
    if os.path.exists(audio_path):
        print(f"File found: {audio_path}")
        try:
            # Process the file
            speech, _ = librosa.load(audio_path, sr=16000)
            # Your processing logic here...
        except Exception as e:
            print(f"Error processing {audio_path}: {str(e)}")
    else:
        print(f"File not found: {audio_path}")



Checking for file: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221077.mp3
File not found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221077.mp3
Checking for file: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221538.mp3
File not found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42221538.mp3


In [22]:
import os
import pandas as pd
import librosa

# Path to your processed_wav directory
processed_wav_path = 'C:\\3CSE80surya\\NM project\\processed_wav'

# List of valid files in the processed_wav folder
valid_files = [
    'common_voice_en_41910645.mp3',
    'common_voice_en_41917474.mp3',
    'common_voice_en_41921421.mp3',
    'common_voice_en_41929259.mp3'
]

# Read your metadata CSV
metadata_path = 'C:\\3CSE80surya\\NM project\\processed_metadata_with_paths.csv'
metadata_df = pd.read_csv(metadata_path)

# Print the first few rows of the metadata to check the data
print("Metadata first few rows:")
print(metadata_df.head())

# Check if 'path' column exists and if it has valid file names
if 'path' in metadata_df.columns:
    print("Paths column found in metadata.")
else:
    print("No 'path' column in metadata.")
    exit()

# Filter the metadata to include only valid files
valid_metadata = metadata_df[metadata_df['path'].isin(valid_files)]

# Print the valid_metadata to check the filtered list
print("Filtered valid metadata:")
print(valid_metadata)

# Process the valid audio files
for index, row in valid_metadata.iterrows():
    audio_path = row['path']
    full_audio_path = os.path.join(processed_wav_path, audio_path)
    
    print(f"Checking file: {full_audio_path}")
    
    if os.path.exists(full_audio_path):
        print(f"File exists: {full_audio_path}")
        try:
            # Load and process the audio
            speech, _ = librosa.load(full_audio_path, sr=16000)
            print(f"Successfully processed {audio_path}")
            # Add your transcription code here
        except Exception as e:
            print(f"Error processing {audio_path}: {str(e)}")
    else:
        print(f"File not found: {full_audio_path}")



Metadata first few rows:
                                                path  \
0  C:\3CSE80surya\NM project\processed_wav\\commo...   
1  C:\3CSE80surya\NM project\processed_wav\\commo...   
2  C:\3CSE80surya\NM project\processed_wav\\commo...   
3  C:\3CSE80surya\NM project\processed_wav\\commo...   
4  C:\3CSE80surya\NM project\processed_wav\\commo...   

                                            sentence                accents  \
0  He was born at Wichenford, in Worcestershire, ...  United States English   
1  The Portuguese division was overrun and withdr...  United States English   
2            Her health by this stage was also poor.                    NaN   
3  His sporting interests outside of cricket incl...        England English   
4  The following year he was elected to be part o...  United States English   

        age gender  
0  thirties    NaN  
1     teens    NaN  
2       NaN    NaN  
3  nineties    NaN  
4     teens    NaN  
Paths column found in metadata.
Filte

In [23]:
import os
import pandas as pd
import librosa

# Path to your processed_wav directory
processed_wav_path = 'C:\\3CSE80surya\\NM project\\processed_wav'

# List of valid files in the processed_wav folder
valid_files = [
    'common_voice_en_41910645.mp3',
    'common_voice_en_41917474.mp3',
    'common_voice_en_41921421.mp3',
    'common_voice_en_41929259.mp3'
]

# Read your metadata CSV
metadata_path = 'C:\\3CSE80surya\\NM project\\processed_metadata_with_paths.csv'
metadata_df = pd.read_csv(metadata_path)

# Print the first few rows of the metadata to check the data
print("Metadata first few rows:")
print(metadata_df.head())

# Check if 'path' column exists and if it has valid file names
if 'path' in metadata_df.columns:
    print("Paths column found in metadata.")
else:
    print("No 'path' column in metadata.")
    exit()

# Clean the paths (remove any double backslashes or extra spaces)
metadata_df['cleaned_path'] = metadata_df['path'].apply(lambda x: os.path.basename(x.strip().replace("\\", "/")))

# Print the cleaned paths to check if they are correct
print("Cleaned paths:")
print(metadata_df['cleaned_path'].head())

# Filter the metadata to include only valid files (matching the valid file names)
valid_metadata = metadata_df[metadata_df['cleaned_path'].isin(valid_files)]

# Print the valid_metadata to check the filtered list
print("Filtered valid metadata:")
print(valid_metadata)

# Process the valid audio files
for index, row in valid_metadata.iterrows():
    audio_path = row['cleaned_path']
    full_audio_path = os.path.join(processed_wav_path, audio_path)
    
    print(f"Checking file: {full_audio_path}")
    
    if os.path.exists(full_audio_path):
        print(f"File exists: {full_audio_path}")
        try:
            # Load and process the audio
            speech, _ = librosa.load(full_audio_path, sr=16000)
            print(f"Successfully processed {audio_path}")
            # Add your transcription code here
        except Exception as e:
            print(f"Error processing {audio_path}: {str(e)}")
    else:
        print(f"File not found: {full_audio_path}")


Metadata first few rows:
                                                path  \
0  C:\3CSE80surya\NM project\processed_wav\\commo...   
1  C:\3CSE80surya\NM project\processed_wav\\commo...   
2  C:\3CSE80surya\NM project\processed_wav\\commo...   
3  C:\3CSE80surya\NM project\processed_wav\\commo...   
4  C:\3CSE80surya\NM project\processed_wav\\commo...   

                                            sentence                accents  \
0  He was born at Wichenford, in Worcestershire, ...  United States English   
1  The Portuguese division was overrun and withdr...  United States English   
2            Her health by this stage was also poor.                    NaN   
3  His sporting interests outside of cricket incl...        England English   
4  The following year he was elected to be part o...  United States English   

        age gender  
0  thirties    NaN  
1     teens    NaN  
2       NaN    NaN  
3  nineties    NaN  
4     teens    NaN  
Paths column found in metadata.
Clean

In [24]:
import os

# List files in the processed_wav directory
processed_wav_path = 'C:\\3CSE80surya\\NM project\\processed_wav'
files_in_directory = os.listdir(processed_wav_path)

# Print the files in the directory to verify they exist
print("Files in processed_wav directory:")
print(files_in_directory)


Files in processed_wav directory:
['common_voice_en_41910645.wav', 'common_voice_en_41910679.wav', 'common_voice_en_41917153.wav', 'common_voice_en_41917154.wav', 'common_voice_en_41917187.wav', 'common_voice_en_41917197.wav', 'common_voice_en_41917205.wav', 'common_voice_en_41917228.wav', 'common_voice_en_41917231.wav', 'common_voice_en_41917242.wav', 'common_voice_en_41917262.wav', 'common_voice_en_41917265.wav', 'common_voice_en_41917266.wav', 'common_voice_en_41917319.wav', 'common_voice_en_41917371.wav', 'common_voice_en_41917375.wav', 'common_voice_en_41917383.wav', 'common_voice_en_41917385.wav', 'common_voice_en_41917391.wav', 'common_voice_en_41917392.wav', 'common_voice_en_41917395.wav', 'common_voice_en_41917459.wav', 'common_voice_en_41917462.wav', 'common_voice_en_41917466.wav', 'common_voice_en_41917468.wav', 'common_voice_en_41917472.wav', 'common_voice_en_41917474.wav', 'common_voice_en_41917478.wav', 'common_voice_en_41917490.wav', 'common_voice_en_41917493.wav', 'comm

In [26]:
import pandas as pd

# Load the CSV file containing the metadata
metadata_path = r'C:\3CSE80surya\NM project\processed_metadata_with_paths.csv'
metadata = pd.read_csv(metadata_path)

# Display the first few rows to confirm it's loaded
print(metadata.head())


                                                path  \
0  C:\3CSE80surya\NM project\processed_wav\\commo...   
1  C:\3CSE80surya\NM project\processed_wav\\commo...   
2  C:\3CSE80surya\NM project\processed_wav\\commo...   
3  C:\3CSE80surya\NM project\processed_wav\\commo...   
4  C:\3CSE80surya\NM project\processed_wav\\commo...   

                                            sentence                accents  \
0  He was born at Wichenford, in Worcestershire, ...  United States English   
1  The Portuguese division was overrun and withdr...  United States English   
2            Her health by this stage was also poor.                    NaN   
3  His sporting interests outside of cricket incl...        England English   
4  The following year he was elected to be part o...  United States English   

        age gender  
0  thirties    NaN  
1     teens    NaN  
2       NaN    NaN  
3  nineties    NaN  
4     teens    NaN  


In [28]:
# Check the columns of the metadata DataFrame
print(metadata.columns)


Index(['path', 'sentence', 'accents', 'age', 'gender'], dtype='object')


In [29]:
# Update the 'path' column to replace '.mp3' with '.wav'
metadata['path'] = metadata['path'].str.replace('.mp3', '.wav')

# Check the updated paths
print(metadata['path'].head())


0    C:\3CSE80surya\NM project\processed_wav\\commo...
1    C:\3CSE80surya\NM project\processed_wav\\commo...
2    C:\3CSE80surya\NM project\processed_wav\\commo...
3    C:\3CSE80surya\NM project\processed_wav\\commo...
4    C:\3CSE80surya\NM project\processed_wav\\commo...
Name: path, dtype: object


In [30]:
# Remove extra backslashes from the 'path' column
metadata['path'] = metadata['path'].str.replace('\\\\', '\\')

# Check the updated paths
print(metadata['path'].head())


0    C:\3CSE80surya\NM project\processed_wav\common...
1    C:\3CSE80surya\NM project\processed_wav\common...
2    C:\3CSE80surya\NM project\processed_wav\common...
3    C:\3CSE80surya\NM project\processed_wav\common...
4    C:\3CSE80surya\NM project\processed_wav\common...
Name: path, dtype: object


In [31]:
# Verify if the files exist
for audio_path in metadata['path']:
    if not os.path.exists(audio_path):
        print(f"File not found: {audio_path}")
    else:
        print(f"File found: {audio_path}")


File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_41923025.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42356358.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42165090.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_41921729.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42528393.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42006138.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42555516.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_41951792.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42446785.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_41974998.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42020013.wav
File found: C:\3CSE80surya\NM project\processed_wav\common_voice_en_42046273.wav
File found: C:\3CSE80surya\N

In [2]:
import os
import pandas as pd
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# -------------------------------
# Step 1: Load and Update Metadata
# -------------------------------

# Change the CSV path below to the actual path of your metadata CSV file.
metadata_csv_path = r'C:\3CSE80surya\NM project\processed_metadata_with_paths.csv'

# Load metadata into a DataFrame
metadata = pd.read_csv(metadata_csv_path)

# Display columns and first few rows for confirmation
print("Metadata columns:", metadata.columns)
print("First few rows of metadata:")
print(metadata.head())

# Update file extensions if needed: replace '.mp3' with '.wav' in the 'path' column.
# (This assumes that the metadata originally had .mp3 references but your files are stored as .wav.)
metadata['path'] = metadata['path'].str.replace('.mp3', '.wav')
# Normalize paths (remove any duplicate backslashes)
metadata['path'] = metadata['path'].str.replace('\\\\', '\\')
print("Updated paths:")
print(metadata['path'].head())

# -------------------------------
# Step 2: Setup the Model and Processor
# -------------------------------

# Use the working alternative model from jonatasgrosman
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# -------------------------------
# Step 3: Define a Transcription Function
# -------------------------------

def transcribe_audio(audio, sampling_rate=16000):
    # Process the audio; note the sampling_rate is passed to satisfy the processor.
    inputs = processor(audio, return_tensors="pt", sampling_rate=sampling_rate, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

# -------------------------------
# Step 4: Process Each Audio File and Transcribe
# -------------------------------

transcriptions = []  # This list will store tuples of (audio_path, transcription)

for idx, row in metadata.iterrows():
    audio_path = row['path']
    
    # Print the path being processed for debugging
    print(f"Processing {audio_path}...")
    
    # Check that the file exists. If you have absolute paths, this should work.
    if not os.path.exists(audio_path):
        print(f"File not found: {audio_path}")
        transcriptions.append((audio_path, None))
        continue

    try:
        # Load the audio (resampled to 16kHz)
        speech, sr = librosa.load(audio_path, sr=16000)
        # Transcribe the audio
        transcription = transcribe_audio(speech, sampling_rate=16000)
        print(f"Transcription for {audio_path}: {transcription}")
        transcriptions.append((audio_path, transcription))
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        transcriptions.append((audio_path, None))

# -------------------------------
# Step 5: Save the Transcriptions to CSV
# -------------------------------

transcriptions_df = pd.DataFrame(transcriptions, columns=['path', 'transcription'])
output_csv_path = r'C:\3CSE80surya\NM project\final_transcriptions.csv'
transcriptions_df.to_csv(output_csv_path, index=False)

print(f"Transcriptions saved to {output_csv_path}")


Metadata columns: Index(['path', 'sentence', 'accents', 'age', 'gender'], dtype='object')
First few rows of metadata:
                                                path  \
0  C:\3CSE80surya\NM project\processed_wav\\commo...   
1  C:\3CSE80surya\NM project\processed_wav\\commo...   
2  C:\3CSE80surya\NM project\processed_wav\\commo...   
3  C:\3CSE80surya\NM project\processed_wav\\commo...   
4  C:\3CSE80surya\NM project\processed_wav\\commo...   

                                            sentence                accents  \
0  He was born at Wichenford, in Worcestershire, ...  United States English   
1  The Portuguese division was overrun and withdr...  United States English   
2            Her health by this stage was also poor.                    NaN   
3  His sporting interests outside of cricket incl...        England English   
4  The following year he was elected to be part o...  United States English   

        age gender  
0  thirties    NaN  
1     teens    NaN  
2      